# FAISS Vector Retrieval System for Sports Injuries Knowledge Base

## Overview
This notebook demonstrates how to use FAISS to retrieve relevant chunks from the sports injuries knowledge base.

**Data Details:**
- **File**: `new_vector_db_dump.sql` (120MB)
- **Records**: 6000+ chunks from 119 PDFs
- **Categories**: 5 sports (badminton, cycling, running, soccer, swimming)
- **Embedding Model**: OpenAI text-embedding-3-small
- **Embedding Dimensions**: 1536
- **Similarity Metric**: Cosine Similarity

## Step 1: Import Required Libraries


In [1]:
import faiss
import numpy as np
import pandas as pd
import re
import json
from typing import List, Dict, Tuple
import os
from openai import OpenAI

print("✓ All libraries imported successfully!")
print(f"FAISS version: {faiss.__version__}")
print(f"NumPy version: {np.__version__}")


✓ All libraries imported successfully!
FAISS version: 1.9.0
NumPy version: 1.26.4


## Step 2: Parse SQL Dump File

Extract embeddings and metadata directly from the PostgreSQL dump file.


In [2]:
def parse_sql_dump(sql_file_path: str) -> Tuple[List[Dict], np.ndarray]:
    """
    Parse PostgreSQL dump file and extract data.
    
    Returns:
        metadata: List of dictionaries with id, category, content, source
        embeddings: NumPy array of shape (n, 1536)
    """
    print(f"Reading SQL dump from: {sql_file_path}")
    
    metadata = []
    embeddings_list = []
    
    with open(sql_file_path, 'r', encoding='utf-8') as f:
        in_copy_section = False
        
        for line in f:
            # Detect start of data section
            if line.startswith('COPY public.knowledge_base'):
                in_copy_section = True
                print("Found data section...")
                continue
            
            # Detect end of data section
            if in_copy_section and line.startswith('\\.'):
                in_copy_section = False
                print("Finished parsing data section")
                break
            
            # Parse data lines
            if in_copy_section and line.strip():
                try:
                    # Split by tabs (PostgreSQL COPY format)
                    parts = line.strip().split('\t')
                    
                    if len(parts) >= 5:
                        record_id = int(parts[0])
                        category = parts[1]
                        content = parts[2]
                        embedding_str = parts[3]
                        source = parts[4] if len(parts) > 4 else ''
                        
                        # Parse embedding: "[0.123, 0.456, ...]" -> list of floats
                        embedding_str = embedding_str.strip('[]')
                        embedding = [float(x) for x in embedding_str.split(',')]
                        
                        if len(embedding) == 1536:
                            metadata.append({
                                'id': record_id,
                                'category': category,
                                'content': content,
                                'source': source
                            })
                            embeddings_list.append(embedding)
                        
                        if len(metadata) % 1000 == 0:
                            print(f"  Parsed {len(metadata)} records...")
                            
                except Exception as e:
                    # Skip problematic lines
                    continue
    
    # Convert to numpy array
    embeddings = np.array(embeddings_list, dtype='float32')
    
    print(f"\n✓ Successfully parsed {len(metadata)} records")
    print(f"✓ Embeddings shape: {embeddings.shape}")
    
    return metadata, embeddings


In [3]:
# Parse the SQL dump file
sql_file = 'new_vector_db_dump_clean.sql'
metadata, embeddings = parse_sql_dump(sql_file)


Reading SQL dump from: new_vector_db_dump_clean.sql
Found data section...
  Parsed 1000 records...
  Parsed 2000 records...
  Parsed 3000 records...
Finished parsing data section

✓ Successfully parsed 3117 records
✓ Embeddings shape: (3117, 1536)


In [4]:
# Display sample data
print("Sample metadata:")
print(f"Total records: {len(metadata)}")
print(f"\nFirst record:")
print(f"  ID: {metadata[0]['id']}")
print(f"  Category: {metadata[0]['category']}")
print(f"  Source: {metadata[0]['source']}")
print(f"  Content preview: {metadata[0]['content'][:200]}...")
print(f"\nEmbedding shape: {embeddings.shape}")
print(f"Embedding sample: {embeddings[0][:5]}...")


Sample metadata:
Total records: 3117

First record:
  ID: 1
  Category: badminton
  Source: IJHSR08.pdf
  Content preview: 1 BPT Intern, Modern College of Physiotherapy, Pune, Maharashtra, India. 2 Associate professor, P.E.S Modern College of Physiotherapy, Pune, Maharashtra, India Corresponding Author: Shantanu Kshirsaga...

Embedding shape: (3117, 1536)
Embedding sample: [-0.04253524 -0.02639948  0.07209279 -0.00048227  0.00677258]...


In [5]:
# Check category distribution
categories = [m['category'] for m in metadata]
category_counts = pd.Series(categories).value_counts()
print("\nCategory distribution:")
print(category_counts)



Category distribution:
soccer       867
cycling      696
swimming     685
badminton    538
running      331
Name: count, dtype: int64


## Step 2.5: Extract Swimming Data for LLM Analysis

Extract swimming-related entries and format them for LLM to create test queries.


In [14]:
def filter_by_category(metadata, embeddings, category='swimming'):
    """
    Filter metadata and embeddings by category.
    
    Args:
        metadata: List of metadata dictionaries
        embeddings: NumPy array of embeddings
        category: Category to filter (default: 'swimming')
    
    Returns:
        Filtered metadata and embeddings
    """
    filtered_metadata = []
    filtered_indices = []
    
    for idx, meta in enumerate(metadata):
        if meta['category'] == category:
            filtered_metadata.append(meta)
            filtered_indices.append(idx)
    
    filtered_embeddings = embeddings[filtered_indices]
    
    print(f"✓ Filtered {len(filtered_metadata)} {category} entries")
    print(f"✓ Filtered embeddings shape: {filtered_embeddings.shape}")
    
    return filtered_metadata, filtered_embeddings

# Filter swimming data
swimming_metadata, swimming_embeddings = filter_by_category(metadata, embeddings, 'swimming')

✓ Filtered 685 swimming entries
✓ Filtered embeddings shape: (685, 1536)


In [16]:
def format_for_llm(filtered_metadata, max_entries=None, output_file=None):
    """
    Format filtered data in a clean, readable format for LLM analysis.
    
    Args:
        filtered_metadata: List of filtered metadata dictionaries
        max_entries: Maximum number of entries to include (None = all)
        output_file: Optional file path to save the formatted text
    
    Returns:
        Formatted string
    """
    entries_to_process = filtered_metadata[:max_entries] if max_entries else filtered_metadata
    
    # Create formatted text (shorter format: only ID and content)
    formatted_text = f"# Swimming Injuries Knowledge Base Entries\n\n"
    formatted_text += f"Total Entries: {len(entries_to_process)}\n"
    formatted_text += f"=" * 80 + "\n\n"
    
    for i, entry in enumerate(entries_to_process, 1):
        formatted_text += f"ID: {entry['id']}\n"
        formatted_text += f"{entry['content']}\n"
    
    # Save to file if specified
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(formatted_text)
        print(f"✓ Saved formatted data to: {output_file}")
    
    return formatted_text

# Format swimming data for LLM
swimming_text = format_for_llm(
    swimming_metadata, 
    max_entries=685,  # Adjust as needed (None = all entries)
    output_file='/Users/apple/Desktop/swimming_entries_for_llm.txt'
)

print(f"✓ Generated {len(swimming_text)} characters of formatted text")
print(f"\nFirst 500 characters preview:")
print(swimming_text[:500])

✓ Saved formatted data to: /Users/apple/Desktop/swimming_entries_for_llm.txt
✓ Generated 643066 characters of formatted text

First 500 characters preview:
# Swimming Injuries Knowledge Base Entries

Total Entries: 685

ID: 6001
<!-- image --> <!-- image --> <!-- image --> At our club we try to get the swimmers 'comfortable being uncomfortable' we train 9 times a week and most of our training is race based training to get the swimmers used to training at high speeds and that definitely makes them uncomfortable most of the time. So its important to get 'comfortable bein


## Step 3: Build FAISS Index

Create a FAISS index using `IndexFlatIP` (Inner Product) for cosine similarity. **Important**: Normalize vectors before adding to the index!


In [8]:
def build_faiss_index(embeddings: np.ndarray) -> faiss.IndexFlatIP:
    """
    Build a FAISS index with normalized embeddings for cosine similarity.
    
    Args:
        embeddings: NumPy array of shape (n, 1536)
    
    Returns:
        FAISS index
    """
    dimension = embeddings.shape[1]
    print(f"Building FAISS index with dimension: {dimension}")
    
    # Create IndexFlatIP (Inner Product) for cosine similarity
    index = faiss.IndexFlatIP(dimension)
    
    # Normalize embeddings for cosine similarity
    print("Normalizing embeddings...")
    faiss.normalize_L2(embeddings)
    
    # Add embeddings to index
    print(f"Adding {len(embeddings)} vectors to index...")
    index.add(embeddings)
    
    print(f"✓ FAISS index built successfully!")
    print(f"  Total vectors in index: {index.ntotal}")
    
    return index

# Build the index
index = build_faiss_index(embeddings)


Building FAISS index with dimension: 1536
Normalizing embeddings...
Adding 3117 vectors to index...
✓ FAISS index built successfully!
  Total vectors in index: 3117


## Step 4: Set Up OpenAI API for Query Embeddings

You need to use the **same model** (`text-embedding-3-small`) to generate query vectors.


In [ ]:
# Set up OpenAI API
# Replace with your actual API key
OPENAI_API_KEY = ""
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Or if you already have it in environment:
client = OpenAI()  # Will use OPENAI_API_KEY from environment

def get_query_embedding(query: str, model: str = "text-embedding-3-small") -> np.ndarray:
    """
    Generate embedding for a query using OpenAI API.
    
    Args:
        query: Query text
        model: OpenAI embedding model (must be text-embedding-3-small)
    
    Returns:
        NumPy array of shape (1536,)
    """
    response = client.embeddings.create(
        input=query,
        model=model
    )
    
    embedding = np.array(response.data[0].embedding, dtype='float32')
    
    # Normalize for cosine similarity
    embedding = embedding.reshape(1, -1)
    faiss.normalize_L2(embedding)
    
    return embedding[0]

print("✓ OpenAI API configured")
print("⚠️ Remember to set your OPENAI_API_KEY environment variable!")


✓ OpenAI API configured
⚠️ Remember to set your OPENAI_API_KEY environment variable!


## Step 5: Perform Similarity Search

Search the FAISS index and retrieve the most relevant chunks.


In [21]:
def search_similar_chunks(
    query: str,
    index: faiss.IndexFlatIP,
    metadata: List[Dict],
    top_k: int = 60,
    category_filter: str = None
) -> List[Dict]:
    """
    Search for similar chunks using FAISS.
    
    Args:
        query: Search query text
        index: FAISS index
        metadata: List of metadata dictionaries
        top_k: Number of results to return
        category_filter: Optional category filter (e.g., 'running', 'soccer')
    
    Returns:
        List of results with content, score, category, and source
    """
    print(f"Query: {query}")
    print(f"Searching for top {top_k} results...")
    
    # Get query embedding
    query_embedding = get_query_embedding(query)
    query_embedding = query_embedding.reshape(1, -1)
    
    # Search FAISS index
    # We search for more results if filtering by category
    search_k = top_k * 10 if category_filter else top_k
    distances, indices = index.search(query_embedding, search_k)
    
    # Prepare results
    results = []
    for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
        if idx >= 0:  # Valid index
            meta = metadata[idx]
            
            # Apply category filter if specified
            if category_filter and meta['category'] != category_filter:
                continue
            
            results.append({
                'rank': len(results) + 1,
                'score': float(distance),  # Cosine similarity score
                'category': meta['category'],
                'content': meta['content'],
                'source': meta['source'],
                'id': meta['id']
            })
            
            if len(results) >= top_k:
                break
    
    print(f"✓ Found {len(results)} results\n")
    return results

def display_results(results: List[Dict]):
    """Pretty print search results."""
    for result in results:
        print(f"{'='*80}")
        print(f"Rank: {result['rank']} | Score: {result['score']:.4f} | Category: {result['category']}")
        print(f"Source: {result['source']}")
        print(f"\nContent:\n{result['content'][:300]}...")
        print()


## Step 6: Test with Sample Queries

Let's test the retrieval system with some example queries about sports injuries.


In [ ]:
# Read test requests from requests.txt
with open('requests.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Split by empty lines and filter out empty strings
queries = [q.strip() for q in content.split('\n\n') if q.strip()]

print(f"✓ Loaded {len(queries)} queries from requests.txt\n")

# Process each query and store results
output_file = 'faiss_retrievals.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("# FAISS Retrieval Results\n")
    f.write("=" * 100 + "\n\n")
    
    for i, query in enumerate(queries, 1):
        print(f"\n{'='*100}")
        print(f"Processing Query {i}/{len(queries)}")
        print(f"{'='*100}")
        
        # Search for similar chunks (top 60 results)
        results = search_similar_chunks(query, index, metadata, top_k=60)
        
        # Write to file
        f.write(f"## QUERY {i}\n")
        f.write(f"{query}\n\n")
        f.write(f"Results: {len(results)} chunks retrieved\n")
        f.write("-" * 100 + "\n\n")
        
        # Write each result
        for result in results:
            f.write(f"Rank: {result['rank']} | Score: {result['score']:.4f} | Category: {result['category']} | ID: {result['id']}\n")
            f.write(f"Source: {result['source']}\n")
            f.write(f"Content: {result['content']}\n")
            f.write("-" * 100 + "\n\n")
        
        f.write("\n" + "=" * 100 + "\n\n")

print(f"\n✓ All {len(queries)} queries processed!")
print(f"✓ Results saved to: {output_file}")


✓ Loaded 16 queries from requests.txt


Processing Query 1/16
Query: What rehabilitation methods are most effective for treating knee injuries in badminton players?
Searching for top 60 results...
✓ Found 60 results


Processing Query 2/16
Query: What preventive strategies are most effective in reducing the incidence of common injuries among badminton players, and how can these be tailored to different player levels and playing styles?
Searching for top 60 results...
✓ Found 60 results


Processing Query 3/16
Query: What are the key factors that influence the recovery time and long-term performance of badminton players after common musculoskeletal injuries?
Searching for top 60 results...
✓ Found 60 results


Processing Query 4/16
Query: Summarize the risk factors for low-back pain in professional cyclists and preventive core exercises supported by evidence.
Searching for top 60 results...
✓ Found 60 results


Processing Query 5/16
Query: Explain how improper saddle height and reach co

In [ ]:
# Summarize retrievals.txt - Extract only IDs for each query
import re

print("Summarizing retrievals.txt...")

with open('faiss_retrievals.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Split by query sections
query_sections = content.split('## QUERY ')

# Skip the header (first split element)
query_sections = query_sections[1:]

# Parse each query section
summary = []
for section in query_sections:
    lines = section.split('\n')
    
    # First line is the query number
    query_num = lines[0].strip()
    
    # Second line is the query text
    query_text = lines[1].strip() if len(lines) > 1 else ''
    
    # Extract all IDs from this section
    ids = []
    for line in lines:
        # Look for lines like "Rank: 1 | Score: 0.7742 | Category: badminton | ID: 356"
        match = re.search(r'\| ID: (\d+)', line)
        if match:
            ids.append(match.group(1))
    
    # Store query number and comma-separated IDs
    summary.append({
        'query_num': query_num,
        'query_text': query_text,
        'ids': ','.join(ids)
    })

# Print summary
print(f"\n{'='*100}")
print("RETRIEVAL SUMMARY - Truth IDs")
print(f"{'='*100}\n")

for item in summary:
    print(f"Query {item['query_num']}: {item['query_text']}")
    print(f"IDs: {item['ids']}")
    print(f"{'-'*100}\n")

# Save summary to file
with open('faiss_retrieval_summary.txt', 'w', encoding='utf-8') as f:
    f.write("# Retrieval Summary - Truth IDs\n")
    f.write("=" * 100 + "\n\n")
    
    for item in summary:
        f.write(f"Query {item['query_num']}: {item['query_text']}\n")
        f.write(f"IDs: {item['ids']}\n\n")

print(f"✓ Summary saved to: retrieval_summary.txt")
print(f"✓ Total queries processed: {len(summary)}")


Summarizing retrievals.txt...

RETRIEVAL SUMMARY - Truth IDs

Query 1: What rehabilitation methods are most effective for treating knee injuries in badminton players?
IDs: 356,371,385,380,376,457,252,377,358,384,375,183,453,440,50,372,290,444,381,362,284,398,360,359,501,512,177,296,387,450,24,208,223,222,379,357,43,172,40,470,166,205,47,369,286,366,238,405,204,333,365,283,331,197,225,181,206,230,282,231
----------------------------------------------------------------------------------------------------

Query 2: What preventive strategies are most effective in reducing the incidence of common injuries among badminton players, and how can these be tailored to different player levels and playing styles?
IDs: 361,405,387,330,409,50,344,208,512,348,332,43,225,358,474,328,26,385,356,333,360,345,243,375,222,380,252,371,205,166,223,241,376,53,410,24,372,329,388,44,207,296,511,384,245,359,206,290,477,429,236,478,522,284,45,381,27,526,436,476
----------------------------------------------------

## Step 7: Save and Load FAISS Index (Optional)

Save the index to disk for reuse without re-parsing the SQL dump.


In [ ]:
# Save FAISS index
index_file = '/Users/apple/Desktop/sports_injuries.faiss'
metadata_file = '/Users/apple/Desktop/sports_injuries_metadata.json'

print("Saving FAISS index and metadata...")
faiss.write_index(index, index_file)

with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"✓ Index saved to: {index_file}")
print(f"✓ Metadata saved to: {metadata_file}")


In [ ]:
# Load FAISS index (for future use)
def load_faiss_index(index_file: str, metadata_file: str):
    """Load saved FAISS index and metadata."""
    print("Loading FAISS index...")
    index = faiss.read_index(index_file)
    
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)
    
    print(f"✓ Loaded index with {index.ntotal} vectors")
    print(f"✓ Loaded {len(metadata)} metadata records")
    
    return index, metadata

# Example usage:
# index, metadata = load_faiss_index(index_file, metadata_file)


## Summary

You've successfully built a FAISS vector retrieval system! Here's what we did:

1. ✓ **Parsed SQL dump** - Extracted 6000+ chunks with embeddings directly from the file
2. ✓ **Built FAISS index** - Created IndexFlatIP with normalized vectors for cosine similarity
3. ✓ **Set up OpenAI API** - Configured query embedding generation with text-embedding-3-small
4. ✓ **Implemented search** - Created functions to search and retrieve relevant chunks
5. ✓ **Tested retrieval** - Verified with sample queries
6. ✓ **Saved index** - Persisted FAISS index and metadata for reuse

### Key Points to Remember:
- ⚠️ Always **normalize vectors** with `faiss.normalize_L2()` for cosine similarity
- ⚠️ Use the **same embedding model** (text-embedding-3-small) for queries
- ⚠️ `IndexFlatIP` is perfect for 6000 records - no need for complex indexes
- ✓ FAISS provides **fast vector search** without needing a database
- ✓ You can optionally **filter by category** in post-processing

### Next Steps:
- Set your `OPENAI_API_KEY` environment variable
- Run the notebook cells in order
- Test with your own queries
- Compare results with PostgreSQL/Elasticsearch if needed
